<a href="https://colab.research.google.com/github/Baek-Donghyeon/Bioinformatics-Algorithms/blob/main/Bioinformatics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bioinformatics

In [ ]:
import numpy
import random
import re
import sys
from collections import defaultdict
from itertools import combinations
sys.setrecursionlimit(3000)

def hamming_distance(p, q):
  if len(p) != len(q):
    raise ValueError('len(p) != len(q)')
  distance = 0
  for i in range(len(p)):
    if p[i] != q[i]:
      distance += 1
  return distance

def pattern_to_number(pattern):
  symbol_to_number = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
  if len(Pattern) == 0:
      return 0
  return 4*pattern_to_number(pattern[:-1]) + symbol_to_number[pattern[-1]]

def number_to_pattern(number, k):
  number_to_symbol = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}
  if k == 1:
    return number_to_symbol(number)
  return number_to_pattern(number // 4, k-1) + number_to_symbol[number % 4]

def count_pattern(text, pattern):
  count = 0
  k = len(pattern)
  for i in range(len(Text)-k+1):
      if text[i:i+k] == pattern:
          count += 1
  return count

def find_frequent_k_mers(text, k, distance = 0, reverse_complement = False):
  ''' k   length of the k_mer '''
  answer = []
  k_mers = frequency_dict(text, k, d, reverse_complement)
  max_frequency = max(k_mers.values())
  for pattern, frequency in k_mers.items():
    if frequency == max_frequency:
      answer.append(pattern)
  return answer

def frequency_dict(text, k, distance = 0, reverse_complement = False):
  ''' k   length of the k_mer '''
  k_mers = defaultdict(int)
  for i in range(len(text) - k + 1):
    for neighbor in neighbors(text[i:i+k], distance):
      k_mers[neighbor] += 1
  if reverse_complement:
    text = reverse_complement(text)
    for i in range(len(text) - k + 1):
      for neighbor in neighbors(text[i:i+k], distance):
        k_mers[neighbor] += 1
  return k_mers

def reverse_complement(text):
  compliment_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}
  answer = []
  for i in text:
    answer += compliment_dict[i]
  return answer[::-1] # reverse the sequence

def find_pattern_occurences(text, pattern, distance = 0):
  # hamming_distance(text, pattern) ≤ d
  positions = []
  k = len(pattern)
  for i in range(len(text) - k + 1):
    if hamming_distance(text[i:i+k], pattern) <= distance:
      positions.append(i)
  return positions

def find_clump(text, k, L, t):
  '''
  k   length of the k_mer
  L   length of an interval
  t   min ocurrences of a pattern in the interval
  '''
  answer = []
  position_dict = defaultdict(list)
  frequency_dict = frequency_dict(text, k)
  for pattern, frequency in frequency_dict.items():
    if frequency >= t:
      position_dict[pattern] = find_occurrences_of_pattern(text, pattern)
  for pattern, position in position_dict.items():
    if max(Position) - min(Position) < L:
      answer.append(pattern)
  return answer

def find_min_skew_position(text):
  # skew : abs(#'G'-#'C')
  answer = []
  skew = [0]
  for base in text:
    if base == 'C':
      skew.append(skew[-1]-1)
    elif base == "G':
      skew.append(skew[-1]+1)
    else:
      skew.append(skew[-1])
  min = min(skew)
  for i in range(len(skew)):
    if skew[i] == min:
      answer.append(i)
  return answer

def neighbors(text, distance):
  answer = []
  length = len(text)
  mismatch_positions = list(combinations(length, distance))
  mismatch_base = [number_to_pattern(number) for number in range(4**k)]
  for positions in mismatch_positions:
    for base in mismatch_base:
      neighbor = text
      for i in distance:
        neighbor(positions[i]) = base[i]
      answer.append(neighbor)
  return answer
      

def enumerate_motif(texts, k, distance):
# Find all (k, distance)-motifs in a collection of strings.
  neighbors_list = []
  for text in texts:
    neighbors_list.append(set(neighbors(text, k, distance)))
  motifs = neighbors_list[0]
  for neighbors in neighbors_list:
    motifs = motifs & neighbors
  return list(motifs)

def DistanceBetweenPatternAndStrings(Pattern, Dna):
#Find the distance between a pattern and a set of strings.
  k = len(Pattern)
  distance = 0
  for string in Dna:
    HamDistance = k
    for i in range(len(string)-k+1):
      CurrentDistance = HammingDistance(Pattern, string[i:i+k])
      if HamDistance > CurrentDistance:
        HamDistance = CurrentDistance
    distance += HamDistance
  return distance


def MedianString(Dna, k):
#Find a median string.
#A median string for Dna minimizes d(Pattern, Dna) over all k-mers Pattern.
  distance = len(Dna)*k
  Median = ''
  for i in range(4**k):
    Pattern = NumberToPattern(i, k)
    CurrentDistance = DistanceBetweenPatternAndStrings(Pattern, Dna)
    if distance > CurrentDistance:
      distance = CurrentDistance
      Median = Pattern
  return Median


def ProfileMostProbableKmer(Text, k, Profile):
#Find a Profile-most probable k-mer in a string.
#Profile-most probable k-mer in Text, a k-mer that was most likely to have been generated by Profile among all k-mers in Text.
  Score = 0
  Kmer = ''
  for i in range(len(Text)-k):
    CurrentScore = 1
    kmer = Text[i:i+k]
    for j in range(k):
      CurrentScore *= Profile[kmer[j]][j]
    if CurrentScore > Score:
      Score = CurrentScore
      Kmer = kmer
  return Kmer


def Score(Motifs, Profile):
  Score = 0
  k = len(Motifs[0])
  for motif in Motifs:
    CurrentScore = 1
    for i in range(k):
      Score *= Profile[motif[i]][i]
    Score += CurrentScore
  return Score


def ProfileFormation(Motifs, k):
  Profile = [[1 for i in range(k)] for j in range(4)]
  for i in range(k):
    for motif in Motifs:
      Profile[motif[i]][i] += 1
  Profile = numpy.array(Profile)/(len(Motifs)+4)
  return Profile


def GreedyMotifSearch(Dna, k, t):
  BestMotifs = []
  for initialmotifs in Dna:
    BestMotifs.append(initialmotifs[:k])
  w = len(Dna[0])
  for x in range(w-k+1):
    Motifs = []
    Motifs.append(Dna[0][x:x+k])
    for y in range(1, t):
      Profile = ProfileFormation(Motifs, k)
      Motifs.append(MostProbableKmer(Dna[y], k, Profile))
    if Score(Motifs, ProfileFormation(Motifs, k)) > Score(Motifs, ProfileFormation(BestMotifs, k)):
      BestMotifs = Motifs
  return BestMotifs


def RandomizedMotifSearch(Dna, k, t):
  w = len(Dna[0])
  # from Symbol to Number representation
  NumDna = [[SymbolToNumber(Symbol) for Symbol in list(line)] for line in Dna]
  Motifs = []
  for i in range(t):
    j = random.randrange(w-k+1)
    Motifs.append(NumDna[i][j:j+k])
  BestMotifs = Motifs
  for x in range(1000):
    while True:
      Profile = ProfileFormation(Motifs, k)
      NewMotifs = []
      for Text in NumDna:
        NewMotifs.append(MostProbableKmer(Text, k, Profile))
      Motifs = NewMotifs
      if Score(Motifs, Profile) > Score(BestMotifs, Profile):
        BestMotifs = Motifs
      else:
        break
    Motifs = []
    for i in range(t):
      j = random.randrange(w-k+1)
      Motifs.append(NumDna[i][j:j+k])
  # from Number to Symbol representation
  return [''.join([NumberToSymbol(Number) for Number in line]) for line in BestMotifs]


def GibbsSampler(Dna, k, t, N):
  w = len(Dna[0])
  # from Symbol to Number representation
  NumDna = [[SymbolToNumber(Symbol) for Symbol in list(line)] for line in Dna]
  Motifs = []
  for i in range(t):
    j = random.randrange(w-k+1)
    Motifs.append(NumDna[i][j:j+k])
  BestMotifs = Motifs
  for j in range(N):
    i = random.randrange(t)
    Motifs.pop(i)
    Profile = ProfileFormation(Motifs, k)
    #Profile-randomly generated k-mer in a string Text.
    Motifs.insert(i, MostProbableKmer(NumDna[i], k, Profile))
    if Score(Motifs, Profile) > Score(BestMotifs, Profile):
      BestMotifs = Motifs
  return [''.join([NumberToSymbol(Number) for Number in line]) for line in BestMotifs]


def Composition(Text, k):
#Generate the k-mer composition of a string.
#k-mer composition is the collection of all k-mer substrings of Text (including repeated k-mers).
  Kmer = []
  for i in range(len(Text)-k+1):
    Kmer.append(Text[i:i+k])
  Kmer.sort()
  #lexicographic order
  return Kmer


def Reconstruct(Patterns):
#Find the string spelled by a genome path.
  Text = Patterns[0]
  for i in range(1,len(Patterns)):
    Text += Patterns[i][-1]
  return Text


def Overlap(Patterns):
#Construct the overlap graph of a collection of k-mers.
#Overlap graph has a node for each k-mer in Patterns and connect k-mers Pattern and Pattern' by a directed edge if Suffix(Pattern) is equal to Prefix(Pattern')
  Kmers = [Patterns.pop()]
  EndOfDna = False
  while not EndOfDna:
    EndOfDna = True
    for kmer in Patterns:
      if Kmers[-1][1:] == kmer[:-1]:
        Kmers.append(kmer)
        Patterns.remove(kmer)
        EndOfDna = False
  EndOfDna = False
  while not EndOfDna:
    EndOfDna = True
    for kmer in Patterns:
      if Kmers[0][:-1] == kmer[1:]:
        Kmers.insert(0, kmer)
        Patterns.remove(kmer)
        EndOfDna = False
  AdjList = []
  for i in range(len(Kmers)-1):
    AdjList.append([Kmers[i], Kmers[i+1]])
  AdjList.sort(key = lambda x: x[0])
  return AdjList

#The de Bruijn graph DeBruijnk(Text) is formed by gluing identically labeled nodes in PathGraphk(Text).
def DeBrujinText(Text, k):
#Construct the de Bruijn graph of a string.
  AdjList = []
  for i in range(len(Text)-k+1):
    AdjList.append([Text[i:i+k-1], Text[i+1:i+k]])
  AdjList.sort(key = lambda x: x[0])
  PathGraph = [AdjList.pop(0)]
  while AdjList:
    ToBeAppended = AdjList.pop(0)
    if ToBeAppended[0] == PathGraph[-1][0]:
      PathGraph[-1].append(ToBeAppended[1])
    else:
      PathGraph.append(ToBeAppended)
  return PathGraph


def DeBrujinPatterns(Patterns, k):
#Construct the de Bruijn graph from a collection of k-mers.
  AdjList = []
  for Kmer in Patterns:
    AdjList.append([Kmer[:-1], Kmer[1:]])
  AdjList.sort(key = lambda x: x[0])
  PathGraph = [AdjList.pop(0)]
  while AdjList:
    ToBeAppended = AdjList.pop(0)
    if ToBeAppended[0] == PathGraph[-1][0]:
      PathGraph[-1].append(ToBeAppended[1])
    else:
      PathGraph.append(ToBeAppended)
  return PathGraph


def DeBrujinPaired(PairedComposition, k):
  AdjList = []
  for Kmer in PairedComposition:
    AdjList.append([[Kmer[0][:-1], Kmer[1][:-1]],[Kmer[0][1:], Kmer[1][1:]]])
  AdjList.sort(key = lambda x: x[0])
  PathGraph = [AdjList.pop(0)]
  while AdjList:
    ToBeAppended = AdjList.pop(0)
    if ToBeAppended[0] == PathGraph[-1][0]:
      PathGraph[-1].append(ToBeAppended[1])
    else:
      PathGraph.append(ToBeAppended)
  return PathGraph


def EulerianPath(PathGraph):
  Graph = []
  AdjDict = {}
  DegreeDict = {}
  for Nodes in PathGraph:
    for Node in Nodes:
      AdjDict.setdefault(Node)
      DegreeDict.setdefault(Node, 0)
  for Nodes in PathGraph:
    AdjDict[Nodes[0]] = Nodes[1:]
    DegreeDict[Nodes[0]] = len(Nodes[1:])

  for EndNodes in AdjDict.values():
    if EndNodes:
      for EndNode in EndNodes:
        DegreeDict[EndNode] = (DegreeDict[EndNode]-1)

  def DFS(index):
    while(AdjDict[index]):
      DFS(AdjDict[index].pop())
    Graph.insert(0, index)

  for index in DegreeDict.keys():
    if DegreeDict[index]==1: # Choosing a start point
      DFS(index)
      break

  return Graph


def DecimalToBinary(Number, k):
  if k == 1:
    return str(Number)
  prefix = Number // 2
  r = Number % 2
  return DecimalToBinary(prefix, k-1) + str(r)


def KUnivercialCircularRing(k):
#A k-universal circular string is a circular string that contains every possible k-mer constructed over a given alphabet.
  AdjDict = {}
  Graph = []
  for i in range(2**(k-1)):
    Binary = DecimalToBinary(i, k-1)
    AdjDict.setdefault(Binary, [Binary[1:]+'1',Binary[1:]+'0'])
  def DFS(index):
    while(AdjDict[index]):
      DFS(AdjDict[index].pop())
    Graph.insert(0, index)
  DFS('0'*(k-1))
  return Graph[:-(k-1)]


def StringReconsturctionFromReadPairs(PathGraph, k, d):
#Reconstruct a string from its paired composition.
#(k,d)-mer is a pair of k-mers in Text separated by distance d.
  AdjDict = {}
  DegreeDict = {}
  PairedGraph = []

  for Pair in PathGraph:
    for Node in Pair:
      AdjDict.setdefault(tuple(Node))
      DegreeDict.setdefault(tuple(Node),0)

  for Pair in PathGraph:
    Prefix = tuple(Pair[0])
    Suffix = Pair[1:]
    AdjDict[Prefix] = Suffix
    DegreeDict[Prefix] = len(Suffix)
  for EndNodes in AdjDict.values():
    if EndNodes:
      for EndNode in EndNodes:
        DegreeDict[tuple(EndNode)] = (DegreeDict[tuple(EndNode)]-1)

  def DFS(index):
    while(AdjDict[index]):
      DFS(tuple(AdjDict[index].pop()))
    PairedGraph.insert(0, list(index))

  for index in DegreeDict.keys():
    if DegreeDict[index]==1:
      DFS(index)
      break

  Graph = []
  for Pair in PairedGraph:
    Graph.append(Pair[0])
  for i in range(-k-d,0):
    Graph.append(PairedGraph[i][1])
  return Graph


def ContigGeneration(PathGraph):
#Generate the contigs from a collection of reads (with imperfect coverage).
#Non-branching if in(v) = out(v) = 1 for each intermediate node v of this path,
  Graph = []
  AdjDict = {}
  DegreeDict = {} # Degree : # of nodes [In, Out]
  for Nodes in PathGraph:
    for Node in Nodes:
      AdjDict.setdefault(Node)
      DegreeDict.setdefault(Node, [0,0])
  for Nodes in PathGraph:
    AdjDict[Nodes[0]] = Nodes[1:]
    DegreeDict[Nodes[0]][1] = len(Nodes[1:])

  for EndNodes in AdjDict.values():
    if EndNodes:
      for EndNode in EndNodes:
        DegreeDict[EndNode][0] = (DegreeDict[EndNode][0]+1)

  def Search(key, contig):
    value = AdjDict[key].pop()
    contig.append(value)
    if DegreeDict[value] == [1,1] and AdjDict[value]:
      return Search(value, contig)
    else: # Finish Search if value is a branch point or has no out
      return Reconstruct(contig)
  
  Contigs = []
  IsNotEmpty = True
  while IsNotEmpty:
    IsNotEmpty = False
    for key in AdjDict.keys():
      if AdjDict[key] and not DegreeDict[key] == [1,1]:
      # key is a branch point
        Contigs.append(Search(key, [key]))
        IsNotEmpty = True

  return Contigs


def ProteinTranslation(Pattern):
  Protein = []
  for i in range(len(Pattern)//3):
    Codon = Pattern[3*i:3*i+3]
    if Codon in ['UAA','UAG','UGA']:
      break
    Protein.append(GeneticCode[Codon])
  return ''.join(Protein)


def DnaToRna(Dna):
  return re.sub('T','U',Dna)


def PeptideEncoding(Text, Peptide):
#Find substrings of a genome encoding a given amino acid sequence.
  Substrings = []
  PeptideLen = len(Peptide)
  for i in range(len(Text)-3*PeptideLen+1):
    substring = Text[i:i+3*PeptideLen]
    if ProteinTranslation(DnaToRna(substring)) == Peptide or ProteinTranslation(DnaToRna(ReverseCompliment(substring))) == Peptide:
      Substrings.append(substring)
  return Substrings


def Cyclospectrum(Peptide):
#Generate the theoretical spectrum of a cyclic peptide.
#The theoretical spectrum of a cyclic peptide Peptide is the collection of all of the masses of its subpeptides
  PeptideLen = len(Peptide)
  TheoreticalSpectrum = [0,Mass(Peptide)]

  Peptide += Peptide
  for i in range(1,PeptideLen):
    for j in range(PeptideLen):
      TheoreticalSpectrum.append(Mass(Peptide[j:j+i]))
  
  return sorted(TheoreticalSpectrum)


def Mass(Peptide):
  Mass = 0
  for AminoAcid in Peptide:
    Mass += IntegerMass[AminoAcid]
  return Mass


def BFCyclopeptideSequencing(Mass):
#Compute the number of peptides of given total mass.
  if Mass in ReducedMass.values():
    global cnt
    cnt += 1
  elif Mass > 57:
    for AminoAcid in ReducedMass.values():
      if Mass > AminoAcid:
        BFCyclopeptideSequencing(Mass-AminoAcid)


def CyclopeptideSequencing(Spectrum):
  
  def Expand(CandidatePeptides):
    Expand = []
    for Peptide in CandidatePeptides:
      for AminoAcid in ReducedMass.keys():
        Expand.append(Peptide+AminoAcid)
    return Expand
  CandidatePeptides = list(ReducedMass.keys())
  FinalPeptides = []
  while True:
    CopyCandidatePeptides = CandidatePeptides.copy()
    for Peptide in CopyCandidatePeptides:
      Peptidespectrum = Cyclospectrum(Peptide)
      if Peptidespectrum == Spectrum:
        FinalPeptides.append(Peptide)
        CandidatePeptides.remove(Peptide)
      else:
        for Fragment in Peptidespectrum:
          if Fragment not in Spectrum:
            CandidatePeptides.remove(Peptide)
            break
    if not CandidatePeptides:
      break
    print(CandidatePeptides)
    CandidatePeptides = Expand(CandidatePeptides)
  print(FinalPeptides)
  return FinalPeptides


GeneticCode = {'AAA':'K','AAC':'N','AAG':'K','AAU':'N','ACA':'T','ACC':'T','ACG':'T','ACU':'T',
               'AGA':'R','AGC':'S','AGG':'R','AGU':'S','AUA':'I','AUC':'I','AUG':'M','AUU':'I',
               'CAA':'Q','CAC':'H','CAG':'Q','CAU':'H','CCA':'P','CCC':'P','CCG':'P','CCU':'P',
               'CGA':'R','CGC':'R','CGG':'R','CGU':'R','CUA':'L','CUC':'L','CUG':'L','CUU':'L',
               'GAA':'E','GAC':'D','GAG':'E','GAU':'D','GCA':'A','GCC':'A','GCG':'A','GCU':'A',
               'GGA':'G','GGC':'G','GGG':'G','GGU':'G','GUA':'V','GUC':'V','GUG':'V','GUU':'V',
               'UAA':'*','UAC':'Y','UAG':'*','UAU':'Y','UCA':'S','UCC':'S','UCG':'S','UCU':'S',
               'UGA':'*','UGC':'C','UGG':'W','UGU':'C','UUA':'L','UUC':'F','UUG':'L','UUU':'F'}

IntegerMass = {'G':57,'A':71,'S':87,'P':97,'V':99,'T':101,'C':103,'I':113,'L':113,'N':114,
               'D':115,'K':128,'Q':128,'E':129,'M':131,'H':137,'F':147,'R':156,'Y':163,'W':186}

ReducedMass = {'G':57,'A':71,'S':87,'P':97,'V':99,'T':101,'C':103,'I':113,'N':114,
               'D':115,'K':128,'E':129,'M':131,'H':137,'F':147,'R':156,'Y':163,'W':186}

f = open('/content/drive/My Drive/Colab Notebooks/input.txt', 'r')
Spectrum = list(map(int,f.readline().split(' ')))
print(CyclopeptideSequencing(Spectrum))
f.close()

['A', 'P', 'V', 'C', 'I', 'N', 'D', 'M', 'H']
['AM', 'AH', 'PV', 'PC', 'PM', 'VP', 'VC', 'VD', 'CP', 'CV', 'CH', 'II', 'IN', 'ID', 'NI', 'NN', 'NM', 'DV', 'DI', 'MA', 'MP', 'MN', 'HA', 'HC']
['PVC', 'PCV', 'VPC', 'VCP', 'CPV', 'CVP', 'III', 'IIN', 'IID', 'INI', 'INN', 'IDI', 'NII', 'NIN', 'NNI', 'DII']
['IIIN', 'IINI', 'INII', 'NIII']
[]
[]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
